# Twiss viewer
### Interactive plot of twiss parameter

**Setup:**
- Install C compiler: `apt install gcc`
- Install apace:`pip install -U apace`

For more info see https://apace.readthedocs.io/en/latest/

In [ ]:
import requests
from ipywidgets import Dropdown
from IPython.display import display

### Select lattice from database

In [ ]:
response = requests.get("https://api.github.com/repos/NoBeam/lattices/contents").json()
menu = Dropdown(description='Lattice:', options=[(item["name"], item["download_url"]) for item in response])
display(menu)

### Fetch lattice file and initilize lattice object

In [ ]:
import apace as ap
url = menu.value
bessy2 = ap.Lattice.from_file(url)
twiss = ap.Twiss(bessy2)

def update_twiss_plot():
    for ax in fig.axes:
        for line, data in zip(ax.lines[3:], (twiss.beta_x, twiss.beta_y, twiss.eta_x * 10)):
            line.set_data(twiss.s, data)

    fig.canvas.draw_idle()

def create_widgets(elements):
    from ipywidgets import BoundedFloatText, Layout, Box, Button
    widgets = []
    for element in elements:
            text = BoundedFloatText(element.k1, description=element.name, min=-5, max=5, step=0.01)
            initial_k1 = element.k1
            button = Button(description=f"{initial_k1:.3f}", icon="undo", disabled=True)
            widgets.append(Box(children=(text, button)))
            
            def on_value_changed(change, element=element, button=button):  # avoid closure late binding
                element.k1 = change.new
                button.disabled = False
                button.button_style = "danger"
                update_twiss_plot()
                
            def on_button_click(button, text=text, element=element, initial_k1=initial_k1):
                text.value = initial_k1
                element.k1 = initial_k1
                button.button_style = ""
                button.disabled = True
            
            text.observe(on_value_changed, "value")
            button.on_click(on_button_click)
            

    layout = Layout(display="grid", grid_template_columns='repeat(3, 1fr)')
    return Box(children=widgets, layout=layout)

### Create interacitve plot

In [ ]:
%matplotlib notebook
from apace.plot import twiss_plot

fig = twiss_plot(twiss, ref_twiss=twiss, sections=(30,60), fig_size=(9,5))
create_widgets(element for element in bessy2.elements if isinstance(element, ap.Quadrupole))

### Export lattice object as LatticeJSON file
- You can use `latticejson autoformat /path/to/file.json` to a cleaned up file

In [ ]:
bessy2.as_dict()